# Examples of @map_e, @fmap_e and map_element
This notebook had examples of map from IoTPy/IoTPy/agent_types/op.py
You can create an agent that maps an input stream to an output stream using map_element, or the decorators @map_e or @fmap_e.
Note: The function map_element and the decorators @map_e and @fmap_e are essentially equivalent. Use the form that you find convenient.

In [1]:
import os
import sys
sys.path.append("../")

In [2]:
from IoTPy.core.stream import Stream, run
from IoTPy.agent_types.op import map_element
from IoTPy.helper_functions.recent_values import recent_values

### Specify streams
<b>x = Stream('x')</b>
<br>
specifies a stream x called 'x'.

### Specify terminating function that is wrapped to create non-terminating agent
<b>def f(v): return 2 * v </b>
<br>
takes a single input argument, <i>v</i> returns a single value --- <i>2*v</i> and terminates.

### Create a non-terminating agent that wraps f and reads stream x and extends stream y.
<b>y[n] = f(x[n]), all n</b>
<br>
<b>map_element(func=f, in_stream=x, out_stream=y)</b>

# Example of map_element

In [3]:
def simple_example_of_map_element():
    # Specify encapsulated functions
    def f(v): return v+10
    
    # Specify streams
    x = Stream('x')
    y = Stream('y')
    
    # Create agent with input stream x and output stream y.
    map_element(func=f, in_stream=x, out_stream=y)
    # y[n] = f(x[n])

    # Put test values in the input streams.
    x.extend(list(range(5)))
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
simple_example_of_map_element()

recent values of stream y are
[10, 11, 12, 13, 14]


## Example illustrating the decorator @map_e
The undecorated function f takes a single argument and returns a single value.
The decorated function f has two arguments, an in_stream and an out_stream, and may have additional keyword arguments.

In [4]:
from IoTPy.agent_types.basics import map_e

def simple_example_of_map_e():

    # Decorate terminating function to specify non-terminating agent.
    @map_e
    def f(v): return v + 10
    
    # Specify streams
    x = Stream(name='x')
    y = Stream(name='y')

    # Create agent with input stream x and output stream y
    f(in_stream=x, out_stream=y)
    # y[n] = x[n]+10

    # Put test values in the input streams.
    x.extend(list(range(5)))
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
    
simple_example_of_map_e()

recent values of stream y are
[10, 11, 12, 13, 14]


## Example illustrating the decorator @fmap_e
This is the functional form of @map_e
The output stream y doesn't have to be declared. f(x) creates and returns a stream.

In [5]:
from IoTPy.agent_types.basics import fmap_e

def simple_example_of_fmap_e():
    # Specify streams
    x = Stream('x')

    # Decorate terminating function to specify non-terminating agent.
    @fmap_e
    def f(v): return v+10
    # Create agent with input stream x and output stream y
    y=f(x)
    # y[n] = x[n]+10

    # Put test values in the input streams.
    x.extend(list(range(5)))
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
simple_example_of_fmap_e()

recent values of stream y are
[10, 11, 12, 13, 14]


## Building Networks of Agents by Connecting Output Streams to Input Streams
You can build networks of agents by connecting the output stream of agents to input streams of agents as shown in the next example. 

In [6]:
def example_of_concatentation_with_map_element():
    # Specify streams.
    x = Stream('x')
    y = Stream('y')
    w = Stream('w')

    # Specify encapsulated functions
    def f(v): return v+10
    def g(w): return w*2

    # Create agent with input stream x and output stream w.
    map_element(func=f, in_stream=x, out_stream=w)
    # y[n] = x[n]+10
    # Create agent with input stream w and output stream y
    map_element(func=g, in_stream=w, out_stream=y)

    # Put test values in the input streams.
    x.extend(list(range(5)))
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
example_of_concatentation_with_map_element()

recent values of stream y are
[20, 22, 24, 26, 28]


## A Network of Agents is an Agent
You can use functions, in the usual way, to specify a function consisting of a network of agents. This function is itself a persistent agent: It reads its input streams and extends its output streams.

In the example below, we specify an agent h with two parameters, x and y, where x is an input stream and y is an output stream. Agent h is composed of two agents --- map_element(f, x, w) and map_element(g, w, y).

In [7]:
def example_of_network_of_agents_is_an_agent():

    # Specify an agent h with is a network of two agents
    # h has an input stream x, and an output stream y.
    def h(x, y):
        # Specify encapsulated functions local to h
        def f(v): return v+10
        def g(w): return w*2
        # Specify an internal stream of h
        w = Stream('w')
        # Specify agents local to h
        map_element(f, x, w)
        map_element(g, w, y)

    # Specify streams.
    x = Stream('x')
    y = Stream('y')

    # Create agent h which is a network of agents
    h(x, y)

    # Put test values in the input streams.
    x.extend(list(range(5)))
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
example_of_network_of_agents_is_an_agent()

recent values of stream y are
[20, 22, 24, 26, 28]


## Function Composition of Agents
You can use fmap_e for a functional form, e.g., g(f(x)) in the exampe below.

In [8]:
def example_of_concatenating_fmap_e():
    # Specify streams
    x = Stream('x')

    # Decorate terminating function to specify non-terminating agent.
    @fmap_e
    def f(v): return v+10
    @fmap_e
    def g(w): return w * 2
    # Create agent with input stream x and output stream y
    y=g(f(x))
    # y[n] = (v+10)*2

    # Put test values in the input streams.
    x.extend(list(range(5)))
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
example_of_concatenating_fmap_e()

recent values of stream y are
[20, 22, 24, 26, 28]


## Keyword arguments of an Agent
(Note: You can also use a class to store keyword arguments and the state.)
In the example below, the function add_constant has two parameters, v and ADDEND where v is an element of the input stream of the agent and ADDEND is a keyword parameter. The function returns a single value which is an element of the output stream of the agent. The call to map_element must have the keyword parameter, ADDEND.

In [9]:
def example_of_keyword_arg_with_map_element():
    # Specify streams
    x = Stream('x')
    y = Stream('y')
    
    # Specify encapsulated functions
    # This function operates on a variable v and has an
    # additional argument ADDEND which will be a keyword
    # argument to specify the agent.
    def add_constant(v, ADDEND): return v + ADDEND

    # Specify agents with keyword arguments
    map_element(func=add_constant, in_stream=x, out_stream=y,
               ADDEND=10)
    # y[n] = x[n] + 10

    # Put test values in the input streams.
    x.extend(list(range(5)))
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
example_of_keyword_arg_with_map_element()

recent values of stream y are
[10, 11, 12, 13, 14]


In [10]:
def example_of_keyword_arg_with_map_e():
    # Specify streams
    x = Stream('x')
    y = Stream('y')

    # Decorate terminating function to specify non-terminating agent.
    @map_e
    def add_constant(v, ADDEND): return v + ADDEND
    # Create agent with input stream x and output stream y with keyword
    # argument
    add_constant(in_stream=x, out_stream=y, ADDEND=10)
    # y[n] = x[n] + 10

    # Put test values in the input streams.
    x.extend(list(range(5)))
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
example_of_keyword_arg_with_map_e()

recent values of stream y are
[10, 11, 12, 13, 14]


## State of an Agent
The agent saves its state between successive calls to its wrapped function.

In the next example, the function f has two arguments, an input element and the state. The function may have additional keyword arguments. The function returns an output element and the next state. The initial state is specified in the call to map_element. In this example, the initial state is 0 because of the call map_element(func=f, in_stream=x, out_stream=y, state=0). Note that the call to map_element must have the keyword argument 'state'.

## Example: map_element with state

In [11]:
def example_of_state_with_map_element():
    # Specify encapsulated functions
    def f(input_element, state):
        next_output = input_element - state
        next_state = input_element
        return next_output, next_state
    
    # Specify streams
    x = Stream(name='x')
    y = Stream(name='y')
    
    # Create agents with input stream x and output stream y
    # and initial state of 0
    map_element(func=f, in_stream=x, out_stream=y, state=0)
    # state[0] = 0, state[n+1] = x[n]
    # y[0] = x[0], y[n+1] = x[n+1] - x[n]

    # Put test values in the input streams.
    x.extend([10, 20, 40, 80])
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
example_of_state_with_map_element()

recent values of stream y are
[10, 10, 20, 40]


## Example of decorator @map_e with state

In [12]:
def example_of_state_with_map_e():
    # Decorate encapsulated functions
    @map_e
    def f(input_element, state):
        next_output = input_element - state
        next_state = input_element
        return next_output, next_state
    
    # Specify streams
    x = Stream(name='x')
    y = Stream(name='y')
    
    # Create agents with input stream x and output stream y
    # and initial state of 0
    f(in_stream=x, out_stream=y, state=0)
    # state[0] = 0, state[n+1] = x[n] - state[n]
    # y[0] = x[0], y[n+1] = x[n+1] - x[n]

    # Put test values in the input streams.
    x.extend([10, 20, 40, 80])
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
example_of_state_with_map_e()

recent values of stream y are
[10, 10, 20, 40]


## Example of function composition using decorator @fmap_e

In [20]:
def example_of_state_with_concatenation_of_fmap_e():
    # Decorate encapsulated functions
    @fmap_e
    def f(input_element, state):
        next_output = input_element - state
        next_state = input_element
        return next_output, next_state
    @fmap_e
    def g(v): return v*2
    
    # Specify streams
    x = Stream('x')
    
    # Create agents with input stream x and output stream y
    # and initial state of 0
    # Example of function composition
    y = g(f(x, state=0))
    # state[0] = 0, state[n+1] = x[n] - state[n]
    # y[0] = x[0], y[n+1] = x[n+1] - x[n]

    # Put test values in the input streams.
    x.extend([10, 20, 40, 80])
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
example_of_state_with_concatenation_of_fmap_e()

recent values of stream y are
[20, 20, 40, 80]


## Agents with both State and Keyword Arguments
The function that is encapsulated can have both state and additional keyword arguments. Note that the call to map_element must have keyword arguments 'state' and the additional keywords. In the following example the call to map_element specifies the initial state (state=0) and the value of the keyword argument (POWER=2).

In [14]:
def example_of_state_with_keyword_arg_with_map_element():
    # Specify streams
    x = Stream(name='x')
    y = Stream(name='y')
    # Specify encapsulated functions
    def f(input_element, state, POWER):
        next_output = input_element**POWER + state
        next_state = input_element + state
        return next_output, next_state
    # Create agents with input stream x and output stream y
    # and initial state of 0, and keyword arg POWER with value 2.
    map_element(func=f, in_stream=x, out_stream=y, state=0, POWER=2)
    # state[0] = 0, state[n+1] = x[0] + ... + x[n]
    # y[0] = x[0]**2,  y[n+1] = x[n+1]**2 + state[n]

    # Put test values in the input streams.
    x.extend(list(range(5)))
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
example_of_state_with_keyword_arg_with_map_element()

recent values of stream y are
[0, 1, 5, 12, 22]


In [15]:
def example_of_state_with_keyword_arg_with_map_e():
    # Specify streams
    x = Stream(name='x')
    y = Stream(name='y')
    # Decorate encapsulated functions
    @map_e
    def f(input_element, state, POWER):
        next_output = input_element**POWER + state
        next_state = input_element + state
        return next_output, next_state
    # Create agents with input stream x and output stream y
    # and initial state of 0, and keyword arg POWER with value 2.
    f(in_stream=x, out_stream=y, state=0, POWER=2)
    # state[0] = 0, state[n+1] = x[0] + ... + x[n]
    # y[0] = x[0]**2,  y[n+1] = x[n+1]**2 + state[n]

    # Put test values in the input streams.
    x.extend(list(range(5)))
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
example_of_state_with_keyword_arg_with_map_e()

recent values of stream y are
[0, 1, 5, 12, 22]


In [16]:
def example_of_state_with_keyword_arg_with_fmap_e():
    # Specify streams
    x = Stream('x')

    # Decorate encapsulated functions
    @fmap_e
    def f(input_element, state, POWER):
        next_output = input_element**POWER + state
        next_state = input_element + state
        return next_output, next_state
    # Create agents with input stream x and output stream y
    # and initial state of 0, and keyword arg POWER with value 2.
    y = f(x, state=0, POWER=2)
    # state[0] = 0, state[n+1] = x[0] + ... + x[n]
    # y[0] = x[0]**2,  y[n+1] = x[n+1]**2 + state[n]

    # Put test values in the input streams.
    x.extend(list(range(5)))
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
example_of_state_with_keyword_arg_with_fmap_e()

recent values of stream y are
[0, 1, 5, 12, 22]


## Saving the State of an Agent in an Argument of the Function
In the following example, the state of the agent is stored in a dict, s. The output of the example is the Fibonacci sequence. In this example, s[0] is the next output of the sequence and s[1] is the element following s[0].

In [17]:
def example_of_saving_state_in_argument():
    # Specify streams
    x = Stream('x')
    y = Stream('y')
    
    s = {'a':0, 'b':1}
    # Specify encapsulated functions
    def f(v, s):
        final, prefinal = s['a'], s['b']
        post_final = final + prefinal
        # In the next state: prefinal becomes final
        # final becomes next_output
        s['a'], s['b'] = post_final, final
        return final

    # Create agent with input stream x and output stream y and
    # keyword argument s
    map_element(f, x, y, s=s)

    # Put test values in the input stream.
    # The values of x aren't relevant in this example
    # because they merely drive the next step of the agent.
    x.extend(list(range(10)))

    # Execute a step
    run()

    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
    assert recent_values(y) == [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

example_of_saving_state_in_argument()

recent values of stream y are
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]


## Storing the State and Arguments of an Agent in a Class
The next example shows how you can save the state and arguments in a class. In this example, the state is running_sum which is the sum of the values read on the input stream, and multiplicand is an argument.

In [18]:
def example_class_to_save_state_and_args():
    class example(object):
        def __init__(self, multiplicand):
            self.multiplicand = multiplicand
            self.running_sum = 0
        def step(self, v):
            result = v * self.multiplicand + self.running_sum
            self.running_sum += v
            return result
    x = Stream()
    y = Stream()
    eg = example(multiplicand=2)
    map_element(func=eg.step, in_stream=x, out_stream=y)

    x.extend(list(range(5)))
    run()
    print ('recent values of stream y are')
    print (recent_values(y))
example_class_to_save_state_and_args()

recent values of stream y are
[0, 2, 5, 9, 14]


In [19]:
from IoTPy.agent_types.op import filter_element
def examples_filter_element():
    x = Stream('x')
    #----------------------------------------------------------------    
    # Filter to only have even numbers
    #----------------------------------------------------------------
    even = Stream()
    filter_element(func=lambda v: not v%2, in_stream=x, out_stream=even)
    # Example: If x = [0, 1, 2, 3, ... ] then y is [0, 2, 4, ...]

    #----------------------------------------------------------------    
    # Filter to only have odd numbers
    #----------------------------------------------------------------
    odd = Stream()
    filter_element(func=lambda v: v%2, in_stream=x, out_stream=odd)

    #----------------------------------------------------------------  
     # Filter to only have negative numbers
    #----------------------------------------------------------------
    neg = Stream('negative')
    filter_element(func=lambda v: v < 0, in_stream=x, out_stream=neg)

    #----------------------------------------------------------------  
     # Filter to only have non_negativenumbers
    #----------------------------------------------------------------
    non_neg = Stream('non_negative')
    filter_element(func=lambda v: v >= 0, in_stream=x, out_stream=non_neg)

    #----------------------------------------------------------------    
    # filter_element with state and no additional arguments
    #----------------------------------------------------------------
    def less_than_n(v, state):
        next_output_element = (v <= state)
        next_state = state+1
        return next_output_element, next_state
    y = Stream('y')
    less = Stream()
    filter_element(func=less_than_n, in_stream=y, out_stream=less, state=0)
    # State on j-th step is j.
    # less_than_n(v, state) returns (v < j) on the j-th step.
    # less filters out all elements v for which v > j
    # So if y is [1, 5, 0, 2, 6, 3] then since states are [ 0, 1, 2, 3, 4,..]
    # then since not(y[0] <= 0), not(y[1] <= 1),
    # y[2] <= 2, y[3] <=3, .... the sequence of outputs of the function
    # less_than_v are [(False, 0), (False, 1), (True, 2), (True, 3), ...]. So
    # the output stream contains y[2], y[3], ... or [0, 2, ...]

    #----------------------------------------------------------------    
    # filter_element with state and with additional keyword arguments
    #----------------------------------------------------------------
    # The keyword argument is addend.
    def less_than_n_plus_addend(v, state, addend):
        # return pair: boolean filter, next state
        return v <= state+addend, state+1
    z = Stream('z')
    less_addend = Stream()
    filter_element(func=less_than_n_plus_addend, in_stream=z, 
                   out_stream=less_addend, state=0, addend=3)
    # State on j-th step is j.
    # Stream less contains z[j] if and only if z[j] <= j+3
    # For example, if z = [2, 3, 3, 4, 10, 15, 7, .....] then the
    # output stream is [2, 3, 3, 4, 7, ...]

    #----------------------------------------------------------------    
    # filter out numbers above the threshold
    #----------------------------------------------------------------
    def threshold(v, threshold): return v > threshold
    above_threshold = Stream('above threshold')
    filter_element(func=threshold, in_stream=x,
                   out_stream=above_threshold, threshold=0)

    # Put data into input streams and run.
    DATA_x = list(range(-5, 5, 1))
    x.extend(DATA_x)
    DATA_y = [1, 5, 0, 2, 6, 3]
    y.extend(DATA_y)
    DATA_z = [2, 3, 3, 4, 10, 15, 7]
    z.extend(DATA_z)
    
    run()

    # Inspect output
    assert recent_values(even) == [-4, -2, 0, 2, 4]
    assert recent_values(odd) == [-5, -3, -1, 1, 3]
    assert recent_values(non_neg) == [0, 1, 2, 3, 4]
    assert recent_values(neg) == [-5, -4, -3, -2, -1]
    assert recent_values(less) == [0, 2, 3]
    assert recent_values(less_addend) == [2, 3, 3, 4, 7]
    assert recent_values(above_threshold) == [1, 2, 3, 4]

    print (recent_values(even))
    print (recent_values(odd))
    print (recent_values(non_neg))
    print (recent_values(neg))
    print (recent_values(less))
    print (recent_values(less_addend))
    print (recent_values(above_threshold))

examples_filter_element()

[-4, -2, 0, 2, 4]
[-5, -3, -1, 1, 3]
[0, 1, 2, 3, 4]
[-5, -4, -3, -2, -1]
[0, 2, 3]
[2, 3, 3, 4, 7]
[1, 2, 3, 4]
